# Calculate robusteness from pi control runs

In [564]:
# Some definitions

# directory of data
#fdir = '~/Google Drive/professional/research/FARALLON_INSTITUTE_PROJECTS/2020 NOAA MAPP/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/'
fdir = '/Volumes/GoogleDrive/My Drive/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/'

In [565]:
# Modules
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore') #filter some warning messages
import seaborn as sns

In [566]:
def num_coevents(y1, y2, pct1, pct2, runs='picontrol',loc='cclme'):
    # 
    fin1 = fdir+'anomaly ts/'+runs+'_'+loc+'_sst_anom.nc'
    fin2 = fdir+'anomaly ts/'+runs+'_sm_anom.nc'
    modsst = xr.open_dataset(fin1)
    modsst.close()
    modsm2 = xr.open_dataset(fin2)
    modsm2.close()
    
    models = modsm2.model

    # selec periood
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))
    
    nev = list()
    
    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values
    
        # calculate threshold
        mhw_thr = np.nanpercentile(tmp1,pct1)
        drg_thr = np.nanpercentile(tmp2,pct2)
    
        a1 = tmp1>=mhw_thr
        a2 = tmp2<=drg_thr
        
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]
    
        nev.append(len(tmp)) #/(y2-y1+1)) #freq.
        
    return nev  # sum(nev)

In [567]:
ny = 30
prc1 = 90
prc2 = 10
loc='goa'

pinev=list()
for i in range(int(500/ny)):
    nev = num_coevents(i*ny, (i+1)*ny-1, prc1 , prc2,loc=loc)
    pinev.append(nev)
pinev,np.mean(pinev) 

([[0, 0, 0, 0, 0, 2, 0, 1, 0, 1, 2, 0, 2, 1, 0, 0, 0, 0, 1, 0, 0, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1],
  [0, 2, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 2, 0, 0, 0, 2, 0],
  [0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0],
  [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 2, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 2, 0, 0],
  [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
  [2, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0],
  [0, 0, 1, 0, 0, 1,

In [568]:
#model by model

In [569]:
model_mean=np.mean(pinev,axis=0)
model_spread=np.std(pinev,axis=0)
model_mean

array([0.25  , 0.4375, 0.25  , 0.4375, 0.1875, 0.4375, 0.125 , 0.125 ,
       0.3125, 0.125 , 0.4375, 0.5625, 0.375 , 0.4375, 0.1875, 0.5625,
       0.375 , 0.25  , 0.375 , 0.4375, 0.1875, 0.25  ])

In [570]:
#16 30-yr periods MMEs

In [571]:
mme_mean=np.mean(pinev,axis=1)
mme_spread=np.std(pinev,axis=1)
#internal variability mean and spread
mme_mean,np.mean(mme_spread)

(array([0.5       , 0.27272727, 0.36363636, 0.22727273, 0.5       ,
        0.40909091, 0.18181818, 0.36363636, 0.22727273, 0.22727273,
        0.18181818, 0.31818182, 0.40909091, 0.36363636, 0.27272727,
        0.36363636]),
 0.5500570308051732)

In [572]:
#MME mean and spread

In [573]:
mme_model_mean=np.mean(mme_mean)
mme_model_spread=np.std(mme_mean)
mme_model_mean,mme_model_spread

(0.32386363636363635, 0.09890281355414325)

In [574]:
#cross model mean and spread

In [575]:
cross_model_mean=np.mean(model_mean)
cross_model_spread=np.std(model_mean)
cross_model_mean,cross_model_spread

(0.32386363636363635, 0.134095724104613)

In [576]:
## Calculate thresholds for PI Control runs

In [577]:
def get_thre(loc,y1,y2,prc1,prc2):
    # open data: anomalies (without trend)

    modsst = xr.open_dataset(fdir+'anomaly ts/picontrol_'+loc+'_sst_anom.nc')
    modsst.close()
    modsm2 = xr.open_dataset(fdir+'anomaly ts/picontrol_sm_anom.nc')
    modsm2.close()

    # select period 
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))
    mhw_thr = list()
    drg_thr = list()
    models = list()

    # Calculate thresholds for each model
    for ix,i in enumerate(modsm2.model.values):
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values
        
        mhw_thr.append(np.nanpercentile(tmp1,prc1))
        drg_thr.append(np.nanpercentile(tmp2,prc2))

        models.append(i)
    
    return mhw_thr, drg_thr , models

In [578]:
# calculate number of events for control, historical and future
# based on pi control thresholds

def freq_coevents(fdir, runs, loc, mhw_thr, drg_thr, models, y1, y2):
    #pi control
    fi1 = fdir+'anomaly ts/'+runs+'_'+loc+'_sst_anom.nc'
    fi2 = fdir+'anomaly ts/'+runs+'_sm_anom.nc'
    modsst = xr.open_dataset(fi1)
    modsst.close()
    modsm2 = xr.open_dataset(fi2)
    modsm2.close()
    models=modsm2.model

    # select period
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))

    nev = list()

    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values

        a1 = tmp1>=mhw_thr[ix]
        a2 = tmp2<=drg_thr[ix]
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]

        nev.append(len(tmp))
    
    freq = np.array(nev) #/(y2-y1+1)
    
    return nev, np.round(np.nanmean(nev),4), np.round(np.nanstd(nev),4) #mean no. of events
#freq, np.round(np.nanmean(freq),4), np.round(np.nanstd(freq),4) #mean frequency


In [579]:
def freq_difruns(loc,ny, y0,y,prc1, prc2):
    
    # calculate threshold in pi control
    #y2=2014
    #y1=y2-ny+1
    mhw_thr, drg_thr, models = get_thre(loc,y0,y,prc1,prc2)

    print ('Percentiles: '+str(prc1)+'/'+str(prc2))
    # pi control
    print('piControl: '+str(y0)+'-'+str(y))
    #y2=2014
    #y1=y2-ny+1
    freq0, mfrq, sfrq = freq_coevents(fdir,'picontrol',loc, mhw_thr,drg_thr, models,y0,y)
    print(mfrq,sfrq)

    # historical 1
    y1 = 1900
    y2 = y1+ny-1
    print('\nhistorical: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'historical',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)

    # historical 2
    y2=2014
    y1=y2-ny+1
    print('\nhistorical 2: '+str(y1)+'-'+str(y2))
    freq1, mfrq, sfrq = freq_coevents(fdir,'historical',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)

    # future
    y2=2099
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq2, mfrq, sfrq = freq_coevents(fdir,'ssp585',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)
    return freq0,freq1,freq2

In [580]:
# Frequency of events - detrended data, using pi control threshols
## percentiles 90-10

In [581]:

print ('\nPercentiles: '+str(prc1)+'/'+str(prc2))
print('\nProbable # events in '+str(ny)+' years'+': '+str(ny*0.01))

funev=list()
hinev=list()
test=list()
for i in range(int(502/ny)):
    pi,hi,fu = freq_difruns(loc,ny,i*ny, (i+1)*ny-1, prc1, prc2)
    test.append(pi)
    funev.append(fu)
    hinev.append(hi)
np.mean(funev),np.mean(hinev),np.std(np.mean(hinev,axis=0))


Percentiles: 90/10

Probable # events in 30 years: 0.3
Percentiles: 90/10
piControl: 0-29
0.5 0.723

historical: 1900-1929
1.0455 0.976

historical 2: 1985-2014
0.8182 0.936

future: 2070-2099
0.5909 0.9844
Percentiles: 90/10
piControl: 30-59
0.2727 0.5378

historical: 1900-1929
1.0455 1.065

historical 2: 1985-2014
0.8182 1.1535

future: 2070-2099
0.6364 1.0679
Percentiles: 90/10
piControl: 60-89
0.3636 0.5677

historical: 1900-1929
1.0 1.0871

historical 2: 1985-2014
0.8636 1.0572

future: 2070-2099
0.4091 0.5768
Percentiles: 90/10
piControl: 90-119
0.2273 0.5163

historical: 1900-1929
0.9091 1.1245

historical 2: 1985-2014
1.0455 1.2605

future: 2070-2099
0.6818 0.762
Percentiles: 90/10
piControl: 120-149
0.5 0.7833

historical: 1900-1929
0.9091 1.0405

historical 2: 1985-2014
1.1818 1.5266

future: 2070-2099
0.5 0.723
Percentiles: 90/10
piControl: 150-179
0.4091 0.6508

historical: 1900-1929
0.8636 1.0994

historical 2: 1985-2014
0.7727 1.1254

future: 2070-2099
0.4091 0.8872
Perc

(0.6960227272727273, 0.9176136363636364, 0.6410956872952703)

In [582]:
np.mean(test),np.mean(pinev)

(0.32386363636363635, 0.32386363636363635)

In [583]:
#model by model

In [584]:
fu_model_mean=np.mean(funev,axis=0)
fu_model_spread=np.std(funev,axis=0)
fu_model_mean

array([0.125 , 0.3125, 1.5   , 0.5625, 0.25  , 0.3125, 0.875 , 0.4375,
       1.0625, 1.0625, 0.375 , 0.9375, 1.3125, 1.1875, 0.25  , 1.0625,
       0.125 , 0.0625, 0.5625, 1.3125, 1.25  , 0.375 ])

In [585]:
np.mean(fu_model_mean),np.std(fu_model_mean)

(0.6960227272727273, 0.4523829953518049)

In [586]:
#mme mean and spread

In [587]:
fu_mme_mean=np.mean(funev,axis=1)
fu_mme_spread=np.std(funev,axis=1)
np.mean(fu_mme_spread)

0.9762132316532354

In [588]:
fu_mme_mean,np.mean(fu_mme_mean),np.std(fu_mme_mean)

(array([0.59090909, 0.63636364, 0.40909091, 0.68181818, 0.5       ,
        0.40909091, 0.81818182, 0.81818182, 1.09090909, 0.77272727,
        0.5       , 0.59090909, 0.95454545, 0.59090909, 1.40909091,
        0.36363636]),
 0.6960227272727273,
 0.2687776268713815)

In [589]:
#16 MMEs diff and spread

In [590]:
diff=list()
zip_obj=zip(fu_mme_mean,mme_mean)
for i , j in zip_obj:
        diff.append(i-j)
diff

[0.09090909090909094,
 0.36363636363636365,
 0.04545454545454547,
 0.4545454545454545,
 0.0,
 0.0,
 0.6363636363636365,
 0.4545454545454546,
 0.8636363636363635,
 0.5454545454545454,
 0.3181818181818182,
 0.27272727272727276,
 0.5454545454545454,
 0.2272727272727273,
 1.1363636363636365,
 0.0]

In [591]:
diff_mean=np.mean(diff)
diff_spread=np.std(diff)
diff_mean,diff_spread

(0.3721590909090909, 0.3166435013094183)

In [592]:
#model by model diff and spread

In [593]:
diff=list()
zip_obj=zip(fu_model_mean,model_mean)
for i , j in zip_obj:
        diff.append(i-j)
diff

[-0.125,
 -0.125,
 1.25,
 0.125,
 0.0625,
 -0.125,
 0.75,
 0.3125,
 0.75,
 0.9375,
 -0.0625,
 0.375,
 0.9375,
 0.75,
 0.0625,
 0.5,
 -0.25,
 -0.1875,
 0.1875,
 0.875,
 1.0625,
 0.125]

In [594]:
diff_mean=np.mean(diff)
diff_spread=np.std(diff)
diff_mean,diff_spread

(0.3721590909090909, 0.45558297783179147)